In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("pre_processed.csv")

/var/folders/kx/vwf66kn17s153dgnqfn48k180000gn/T/ipykernel_13783/2583887785.py:1: DtypeWarning: Columns (2,7,9,14,15,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("pre_processed.csv")


In [3]:
df.head()

,created_time,comment_id,score,self_text,subreddit,post_id,author_name,controversiality,ups,user_is_verified,...,user_total_karma,post_score,post_self_text,post_title,post_upvote_ratio,post_thumbs_ups,post_created_time,post_title_tokens,self_text_tokens,post_self_text_tokens
0,2024-03-19 19:18:08,kvmer6g,1,What is this supposed to prove? That nobody in...,IsraelPalestine,1biqebd,tinamnstrrr,0,1.0,True,...,5371.0,0.0,"So, Mr. Netanyahu, and supporters, are you goo...",article: Famine in northern Gaza is imminent a...,0.36,0,2024-03-19 17:48:02,"['article', 'famine', 'in', 'northern', 'gaza'...","['what', 'is', 'this', 'supposed', 'to', 'prov...","['so', 'netanyahu', 'and', 'supporters', 'are'..."
1,2024-03-19 19:17:51,kvmepe5,1,If only there was an armed military force near...,worldnews,1bik6xp,seclifered,0,1.0,True,...,4556.0,685.0,NaN,Local Gaza gangs are ‘robbing displaced Palest...,0.89,685,2024-03-19 13:25:00,"['local', 'gaza', 'gangs', 'are', 'robbing', '...","['if', 'only', 'there', 'was', 'an', 'armed', ...","['nan', 'nan']"
2,2024-03-19 19:17:35,kvmenpq,1,Taliban created and funded by america and isis...,IsraelPalestine,1biqbsz,Accomplished_Mud6174,0,1.0,True,...,-32.0,14.0,Most pro Palestine supporters who are not Arab...,Why is the West so concerned?,0.75,14,2024-03-19 17:45:08,"['why', 'is', 'the', 'west', 'so', 'concerned'...","['taliban', 'created', 'and', 'funded', 'by', ...","['most', 'pro', 'palestine', 'supporters', 'wh..."
3,2024-03-19 19:17:25,kvmemm5,1,Recycling glass is still not economical or par...,worldnews,1biqh0j,lordraiden007,0,1.0,True,...,38228.0,519.0,NaN,The plastic industry knowingly pushed recyclin...,0.97,519,2024-03-19 17:51:10,"['the', 'plastic', 'industry', 'knowingly', 'p...","['recycling', 'glass', 'is', 'still', 'not', '...","['nan', 'nan']"
4,2024-03-19 19:17:22,kvmem94,1,The two have a private direct communication li...,worldnews,1bis6xt,MajorTechnology8827,0,1.0,True,...,12478.0,0.0,NaN,Biden privately told Netanyahu he's not trying...,0.5,0,2024-03-19 19:00:41,"['biden', 'privately', 'told', 'netanyahu', 'h...","['the', 'two', 'have', 'a', 'private', 'direct...","['nan', 'nan']"


In [4]:
import numpy as np
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

# Initialize NLTK English stop words
nltk_stop_words = set(stopwords.words('english'))


# Load GloVe embeddings
def load_glove_embeddings(file_path):
    embeddings_index = {}
    with open(file_path, encoding="utf-8") as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    return embeddings_index

# Path to the downloaded GloVe embeddings
glove_embeddings_path = '/Users/arafathjazeeb/Library/CloudStorage/OneDrive-UniversityofPisa/Erasmus/Text Analytics/Project/glove.6B/glove.6B.300d.txt'  # Replace with the actual path to your GloVe file

# Load GloVe embeddings
glove_embeddings = load_glove_embeddings(glove_embeddings_path)

# Function to remove top 25% most frequent words that match with NLTK English stop words
def remove_top_25_percent(words):
    total_words = len(words)
    top_25_percent_count = int(total_words * 0.25)
    stop_words_set = set(stopwords.words('english'))
    
    # Filter out the words that are both in the top 5% and are NLTK English stop words
    top_25_percent_words = [word for word, score in words[:top_25_percent_count] if word in stop_words_set]
    return top_25_percent_words

# Function to compute GloVe scores for a given column
def compute_glove_scores(column):
    column = column.fillna('')
    # Initialize CountVectorizer
    count_vectorizer = CountVectorizer()
    X = count_vectorizer.fit_transform(column)
    
    # Initialize TfidfTransformer
    tfidf_transformer = TfidfTransformer()
    X = tfidf_transformer.fit_transform(X)
    
    # Calculate GloVe scores
    feature_names = count_vectorizer.get_feature_names_out()
    word_scores = {}
    for word in feature_names:
        if word in glove_embeddings:
            word_scores[word] = np.mean(glove_embeddings[word])
        else:
            word_scores[word] = 0.0

    sorted_words_scores = sorted(word_scores.items(), key=lambda x: x[1], reverse=True)
    return sorted_words_scores


In [5]:
#Glove Score Calculating
words_title_gscore = compute_glove_scores(df['post_title_tokens'])
self_text_gscore = compute_glove_scores(df['self_text_tokens'])
post_self_text_gscore = compute_glove_scores(df['post_self_text_tokens'])

In [6]:
#stopwords removed (from) top 5%frequent words in each column
removed_words_title = remove_top_25_percent(words_title_gscore)
removed_words_self = remove_top_25_percent(self_text_gscore)
removed_words_post_self = remove_top_25_percent(post_self_text_gscore)

In [7]:
# File paths to save the results
output_removed_words_paths = ['removed_words_title.txt', 'removed_words_self.txt', 'removed_words_post_self.txt']
output_glove_scores_paths = ['words_title_gscore.txt', 'self_text_gscore.txt', 'post_self_text_gscore.txt']

# Lists of removed words and corresponding GloVe scores
removed_words_lists = [removed_words_title, removed_words_self, removed_words_post_self]
glove_scores_lists = [words_title_gscore, self_text_gscore, post_self_text_gscore]

# Iterate over each list of removed words and corresponding GloVe scores
for removed_words_path, glove_scores_path, removed_words_list, glove_scores_list in zip(output_removed_words_paths, output_glove_scores_paths, removed_words_lists, glove_scores_lists):
    # Open the file for removed words in write mode
    with open(removed_words_path, 'w') as file:
        # Write the removed words to the file
        file.write('Removed words:\n')
        file.write(', '.join(removed_words_list) + '\n\n')
    print("Removed words for", removed_words_path, "have been saved.")

    # Open the file for GloVe scores in write mode
    with open(glove_scores_path, 'w') as file:
        # Write the GloVe scores to the file
        file.write('GloVe Scores:\n')
        for word, score in glove_scores_list:
            file.write(f'{word}: {score}\n')
    print("GloVe scores for", glove_scores_path, "have been saved.")

print("All results have been saved.")


Removed words for removed_words_title.txt have been saved.
GloVe scores for words_title_gscore.txt have been saved.
Removed words for removed_words_self.txt have been saved.
GloVe scores for self_text_gscore.txt have been saved.
Removed words for removed_words_post_self.txt have been saved.
GloVe scores for post_self_text_gscore.txt have been saved.
All results have been saved.


In [8]:
#Stopwords Removal
import ast
# Replace NaN values with an empty list
df["self_text_tokens"].fillna('[]', inplace=True)
df["post_title_tokens"].fillna('[]', inplace=True)
df["post_self_text_tokens"].fillna('[]', inplace=True)

/var/folders/kx/vwf66kn17s153dgnqfn48k180000gn/T/ipykernel_13783/2135951388.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["self_text_tokens"].fillna('[]', inplace=True)
/var/folders/kx/vwf66kn17s153dgnqfn48k180000gn/T/ipykernel_13783/2135951388.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values alw

In [9]:
df["self_text_tokens"] = df["self_text_tokens"].apply(lambda x: [word for word in ast.literal_eval(x) if word not in removed_words_self])

In [10]:
df["post_title_tokens"] = df["post_title_tokens"].apply(lambda x: [word for word in ast.literal_eval(x) if word not in removed_words_title])

In [11]:
import ast

batch_size = 10000  # Define the batch size

# Determine the number of batches
num_batches = (len(df) + batch_size - 1) // batch_size

removed_words_post_self = [...]  # Define your list of removed words

# Iterate over batches
for i in range(num_batches):
    start_idx = i * batch_size
    end_idx = min((i + 1) * batch_size, len(df))
    
    # Process a batch of data
    batch = df["post_self_text_tokens"].iloc[start_idx:end_idx].apply(
        lambda x: [word for word in ast.literal_eval(x) if word not in removed_words_post_self]
    )
    
    # Update the dataframe with processed data
    df["post_self_text_tokens"].iloc[start_idx:end_idx] = batch

    # Print the batch number
    print(f"Processed batch {i+1} of {num_batches}")


/var/folders/kx/vwf66kn17s153dgnqfn48k180000gn/T/ipykernel_13783/1375079519.py:21: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df["post_self_text_tokens"].iloc[start_idx:end_idx] = batch
/var/folders/kx/vwf66kn17s153dgnqfn48k180000gn/T/ipy

Processed batch 1 of 109


/var/folders/kx/vwf66kn17s153dgnqfn48k180000gn/T/ipykernel_13783/1375079519.py:21: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df["post_self_text_tokens"].iloc[start_idx:end_idx] = batch
/var/folders/kx/vwf66kn17s153dgnqfn48k180000gn/T/ipy

Processed batch 2 of 109


/var/folders/kx/vwf66kn17s153dgnqfn48k180000gn/T/ipykernel_13783/1375079519.py:21: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df["post_self_text_tokens"].iloc[start_idx:end_idx] = batch
/var/folders/kx/vwf66kn17s153dgnqfn48k180000gn/T/ipy

...

/var/folders/kx/vwf66kn17s153dgnqfn48k180000gn/T/ipykernel_13783/1375079519.py:21: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df["post_self_text_tokens"].iloc[start_idx:end_idx] = batch
/var/folders/kx/vwf66kn17s153dgnqfn48k180000gn/T/ipy

Processed batch 107 of 109


/var/folders/kx/vwf66kn17s153dgnqfn48k180000gn/T/ipykernel_13783/1375079519.py:21: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df["post_self_text_tokens"].iloc[start_idx:end_idx] = batch
/var/folders/kx/vwf66kn17s153dgnqfn48k180000gn/T/ipy

Processed batch 108 of 109
Processed batch 109 of 109


/var/folders/kx/vwf66kn17s153dgnqfn48k180000gn/T/ipykernel_13783/1375079519.py:21: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df["post_self_text_tokens"].iloc[start_idx:end_idx] = batch
/var/folders/kx/vwf66kn17s153dgnqfn48k180000gn/T/ipy

In [12]:
df.to_csv('pre_processed.csv', index=False)

In [14]:
len(self_text_gscore)

180065

In [15]:
print(len(removed_words_title))

9


: 